In [1]:
import pandas as pd
import numpy as np
import warnings

noaa_file_path = "./data/NOAA_StormEvents/StormEvents_2014_2024.csv"
# [Previous loading code remains the same]
# ...
try:
    df_noaa = pd.read_csv(noaa_file_path, low_memory=False)
    print("NOAA data loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {noaa_file_path}")
    exit()
except Exception as e:
    print(f"Error loading NOAA data: {e}")
    exit()

print("Performing initial cleaning and type conversions...")

# --- Datetime Conversion ---
noaa_datetime_format = '%d-%b-%y %H:%M:%S'
df_noaa['BEGIN_DT'] = pd.to_datetime(df_noaa['BEGIN_DATE_TIME'], format=noaa_datetime_format, errors='coerce')
df_noaa['END_DT'] = pd.to_datetime(df_noaa['END_DATE_TIME'], format=noaa_datetime_format, errors='coerce')

begin_nat_count = df_noaa['BEGIN_DT'].isna().sum()
end_nat_count = df_noaa['END_DT'].isna().sum()
# ... (rest of initial datetime parsing checks) ...

# --- *** INSPECT TIMEZONES *** ---
print("\n--- Unique Timezones Found in CZ_TIMEZONE ---")
unique_timezones = df_noaa['CZ_TIMEZONE'].unique()
print(unique_timezones)
print(f"Number of unique timezone entries: {len(unique_timezones)}")
# --- *** END INSPECTION *** ---


NOAA data loaded successfully.
Performing initial cleaning and type conversions...

--- Unique Timezones Found in CZ_TIMEZONE ---
['EST-5' 'CST-6' 'PST-8' 'MST-7' 'HST-10' 'AKST-9' 'AST-4' 'GST10'
 'SST-11' 'PDT-7' 'CDT-5' 'EDT-4']
Number of unique timezone entries: 12


In [2]:
import pandas as pd
import numpy as np
import warnings

# Define the path to the NOAA data file
noaa_file_path = 'data/NOAA_StormEvents/StormEvents_2014_2024.csv'

# Define columns that are likely numeric but might have issues during load
numeric_cols_to_check = [
    'BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME',
    'EPISODE_ID', 'EVENT_ID', 'STATE_FIPS', 'CZ_FIPS',
    'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT',
    'MAGNITUDE', 'TOR_LENGTH', 'TOR_WIDTH',
    'BEGIN_RANGE', 'END_RANGE', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON'
]

# Define the expected datetime format
# Using errors='coerce' will turn unparseable dates into NaT (Not a Time)
noaa_datetime_format = '%d-%b-%y %H:%M:%S'

print(f"Loading NOAA data from: {noaa_file_path}")

# --- Load the data ---
# Consider using low_memory=False if dtype warnings appear, or specify dtypes more precisely
# For very large files, consider chunking or libraries like Dask/Polars
try:
    df_noaa = pd.read_csv(noaa_file_path, low_memory=False)
    print("NOAA data loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {noaa_file_path}")
    # Exit or handle error appropriately
    exit()
except Exception as e:
    print(f"Error loading NOAA data: {e}")
    exit()

print("Performing initial cleaning and type conversions...")

# --- Datetime Conversion ---
# Convert BEGIN_DATE_TIME and END_DATE_TIME
# errors='coerce' handles unparseable formats by setting them to NaT
df_noaa['BEGIN_DT'] = pd.to_datetime(df_noaa['BEGIN_DATE_TIME'], format=noaa_datetime_format, errors='coerce')
df_noaa['END_DT'] = pd.to_datetime(df_noaa['END_DATE_TIME'], format=noaa_datetime_format, errors='coerce')

# Check for parsing errors (NaT values)
begin_nat_count = df_noaa['BEGIN_DT'].isna().sum()
end_nat_count = df_noaa['END_DT'].isna().sum()
if begin_nat_count > 0 or end_nat_count > 0:
    print(f"Warning: Found {begin_nat_count} NaT values in BEGIN_DT after parsing.")
    print(f"Warning: Found {end_nat_count} NaT values in END_DT after parsing.")
    # Consider dropping or investigating rows with NaT datetimes if they are critical
    # df_noaa.dropna(subset=['BEGIN_DT', 'END_DT'], inplace=True)


# --- Timezone Handling ---
# Map common timezone abbreviations to standard Olson names usable by pandas
# This might need expansion based on unique values in CZ_TIMEZONE
tz_map = {
    # Standard US Timezones (using Olson names that handle DST)
    'EST-5': 'America/New_York',    # Eastern Time
    'EDT-4': 'America/New_York',    # Eastern Time (Daylight)
    'CST-6': 'America/Chicago',     # Central Time
    'CDT-5': 'America/Chicago',     # Central Time (Daylight)
    'MST-7': 'America/Denver',      # Mountain Time (most areas)
    'MDT-6': 'America/Denver',      # Mountain Time (most areas - Daylight) - Added MDT just in case although not in list
    'PST-8': 'America/Los_Angeles', # Pacific Time
    'PDT-7': 'America/Los_Angeles', # Pacific Time (Daylight)
    'AKST-9': 'America/Anchorage',  # Alaska Time
    'AKDT-8': 'America/Anchorage',  # Alaska Time (Daylight) - Added AKDT just in case
    'HST-10': 'Pacific/Honolulu',   # Hawaii Standard Time (no DST)

    # Atlantic & Territories
    'AST-4': 'America/Puerto_Rico', # Atlantic Standard Time (no DST in PR)
    'GST10': 'Pacific/Guam',        # Guam Standard Time (UTC+10)
    'SST-11': 'Pacific/Pago_Pago',   # Samoa Standard Time (UTC-11)

    # Add mappings for any potential NaN or empty strings if they exist
    '': None, # Map empty string explicitly if needed
    # np.nan: None # pd.isna() check in function should handle actual NaN objects
}

# Function to apply timezone localization
def localize_datetime(row):
    tz_str = row['CZ_TIMEZONE']
    dt = row['datetime_col']
    if pd.isna(dt) or pd.isna(tz_str):
        return pd.NaT

    tz_name = tz_map.get(tz_str)
    if tz_name:
        try:
            # Localize the naive datetime
            return dt.tz_localize(tz_name, ambiguous='NaT', nonexistent='NaT')
        except Exception as e:
            # Log warning for specific row/error if needed
            # warnings.warn(f"Could not localize timezone '{tz_str}' for datetime {dt}: {e}")
            return pd.NaT # Failed to localize
    else:
        # Log warning for unmapped timezone if needed
        # warnings.warn(f"Timezone '{tz_str}' not found in tz_map.")
        return pd.NaT # Timezone not in map

# Apply localization - requires iterating or a more complex apply
# Create temporary column for the function
df_noaa['datetime_col'] = df_noaa['BEGIN_DT']
df_noaa['BEGIN_DT_LOC'] = df_noaa.apply(localize_datetime, axis=1)

df_noaa['datetime_col'] = df_noaa['END_DT']
df_noaa['END_DT_LOC'] = df_noaa.apply(localize_datetime, axis=1)

unique_dt = df_noaa['BEGIN_DT_LOC'].unique()

print(unique_dt)

Loading NOAA data from: data/NOAA_StormEvents/StormEvents_2014_2024.csv
NOAA data loaded successfully.
Performing initial cleaning and type conversions...
[Timestamp('2014-02-18 10:00:00-0500', tz='America/New_York')
 Timestamp('2014-03-30 08:31:00-0400', tz='America/New_York')
 Timestamp('2014-04-27 23:06:00-0500', tz='America/Chicago') ...
 Timestamp('2024-05-09 12:53:00-0500', tz='America/Chicago')
 Timestamp('2024-05-22 18:09:00-0400', tz='America/New_York')
 Timestamp('2024-08-06 07:52:00-0400', tz='America/New_York')]


In [8]:
df_noaa['BEGIN_DT_LOC']

0         2014-02-18 10:00:00-05:00
1         2014-03-30 08:31:00-04:00
2         2014-04-27 23:06:00-05:00
3         2014-04-27 23:03:00-05:00
4         2014-02-15 13:00:00-08:00
                    ...            
691429    2024-05-26 11:48:00-04:00
691430    2024-05-22 18:09:00-04:00
691431    2024-05-22 17:57:00-04:00
691432    2024-06-23 17:45:00-04:00
691433    2024-08-06 07:52:00-04:00
Name: BEGIN_DT_LOC, Length: 691434, dtype: object

In [3]:
import pandas as pd
import numpy as np
import warnings

# Define the path to the NOAA data file
noaa_file_path = 'data/NOAA_StormEvents/StormEvents_2014_2024.csv'

# Define columns that are likely numeric but might have issues during load
numeric_cols_to_check = [
    'BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME',
    'EPISODE_ID', 'EVENT_ID', 'STATE_FIPS', 'CZ_FIPS',
    'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT',
    'MAGNITUDE', 'TOR_LENGTH', 'TOR_WIDTH',
    'BEGIN_RANGE', 'END_RANGE', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON'
]

# Define the expected datetime format
# Using errors='coerce' will turn unparseable dates into NaT (Not a Time)
noaa_datetime_format = '%d-%b-%y %H:%M:%S'

print(f"Loading NOAA data from: {noaa_file_path}")

# --- Load the data ---
try:
    df_noaa = pd.read_csv(noaa_file_path, low_memory=False)
    print("NOAA data loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {noaa_file_path}")
    exit()
except Exception as e:
    print(f"Error loading NOAA data: {e}")
    exit()

print("Performing initial cleaning and type conversions...")

# --- Datetime Conversion (Initial Parsing) ---
print("Parsing original datetime strings...")
df_noaa['BEGIN_DT'] = pd.to_datetime(df_noaa['BEGIN_DATE_TIME'], format=noaa_datetime_format, errors='coerce')
df_noaa['END_DT'] = pd.to_datetime(df_noaa['END_DATE_TIME'], format=noaa_datetime_format, errors='coerce')

# Check for initial parsing errors (NaT values)
begin_nat_count = df_noaa['BEGIN_DT'].isna().sum()
end_nat_count = df_noaa['END_DT'].isna().sum()
if begin_nat_count > 0 or end_nat_count > 0:
    print(f"Warning: Found {begin_nat_count} NaT values in BEGIN_DT after initial parsing.")
    print(f"Warning: Found {end_nat_count} NaT values in END_DT after initial parsing.")

# --- Timezone Handling ---
print("Mapping timezones...")
# Using the tz_map confirmed from your data
tz_map = {
    'EST-5': 'America/New_York',
    'EDT-4': 'America/New_York',
    'CST-6': 'America/Chicago',
    'CDT-5': 'America/Chicago',
    'MST-7': 'America/Denver',
    'MDT-6': 'America/Denver', # Keep just in case
    'PST-8': 'America/Los_Angeles',
    'PDT-7': 'America/Los_Angeles',
    'AKST-9': 'America/Anchorage',
    'AKDT-8': 'America/Anchorage', # Keep just in case
    'HST-10': 'Pacific/Honolulu',
    'AST-4': 'America/Puerto_Rico',
    'GST10': 'Pacific/Guam',
    'SST-11': 'Pacific/Pago_Pago',
    '': None, # Map empty string explicitly if needed
}

# Function to apply timezone localization
def localize_datetime(row):
    tz_str = row['CZ_TIMEZONE']
    dt = row['datetime_col']
    if pd.isna(dt) or pd.isna(tz_str):
        return pd.NaT

    # Ensure tz_str is string and strip whitespace for lookup
    tz_str = str(tz_str).strip()
    tz_name = tz_map.get(tz_str)

    if tz_name:
        try:
            return dt.tz_localize(tz_name, ambiguous='NaT', nonexistent='NaT')
        except Exception as e:
            # warnings.warn(f"Could not localize timezone '{tz_str}' for datetime {dt}: {e}")
            return pd.NaT
    else:
        if tz_str: # Avoid warning for known blanks mapped to None
             warnings.warn(f"Timezone '{tz_str}' not found in tz_map.", UserWarning)
        return pd.NaT

# Apply localization using the helper column
print("Applying timezone localization (this may take time)...")
df_noaa['datetime_col'] = df_noaa['BEGIN_DT']
df_noaa['BEGIN_DT_LOC'] = df_noaa.apply(localize_datetime, axis=1)

df_noaa['datetime_col'] = df_noaa['END_DT']
df_noaa['END_DT_LOC'] = df_noaa.apply(localize_datetime, axis=1)

df_noaa.drop(columns=['datetime_col'], inplace=True)
print("Timezone localization applied.")

# --- Convert Mixed Timezone 'Object' Columns Directly to UTC ---
# This step addresses the 'dtype: object' issue after localization
print("Attempting to convert localized columns directly to UTC...")
try:
    # Record NaNs before conversion
    original_loc_nan_begin = df_noaa['BEGIN_DT_LOC'].isna().sum()
    original_loc_nan_end = df_noaa['END_DT_LOC'].isna().sum()

    # Use pd.to_datetime with utc=True to handle the object column containing tz-aware objects
    df_noaa['BEGIN_DT_UTC'] = pd.to_datetime(df_noaa['BEGIN_DT_LOC'], errors='coerce', utc=True)
    df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_LOC'], errors='coerce', utc=True)

    print("Direct UTC conversion attempted.")

    # Check for new NaNs potentially introduced
    begin_utc_nat_count = df_noaa['BEGIN_DT_UTC'].isna().sum()
    end_utc_nat_count = df_noaa['END_DT_UTC'].isna().sum()
    if begin_utc_nat_count > original_loc_nan_begin or end_utc_nat_count > original_loc_nan_end:
         print(f"Warning: Additional NaTs potentially introduced during UTC conversion.")
         print(f"         ({original_loc_nan_begin} -> {begin_utc_nat_count} NaTs in BEGIN_DT_UTC)")
         print(f"         ({original_loc_nan_end} -> {end_utc_nat_count} NaTs in END_DT_UTC)")

except Exception as e:
    print(f"ERROR: Failed to convert localized object columns to UTC. Error: {e}")
    # Assign NaT if conversion fails catastrophically
    df_noaa['BEGIN_DT_UTC'] = pd.NaT
    df_noaa['END_DT_UTC'] = pd.NaT


# --- Numeric Conversions ---
print("Converting numeric columns...")
for col in numeric_cols_to_check:
    if col in df_noaa.columns:
        df_noaa[col] = pd.to_numeric(df_noaa[col], errors='coerce')
        if 'INJURIES' in col or 'DEATHS' in col:
            # Fill NaNs only for specific columns where 0 makes sense
            df_noaa[col].fillna(0, inplace=True)


# --- FIPS Code to String ---
print("Converting FIPS codes to string...")
if 'STATE_FIPS' in df_noaa.columns:
    # Use .astype(str).str.split('.').str[0] to handle potential floats before converting to string
    df_noaa['STATE_FIPS'] = df_noaa['STATE_FIPS'].astype(str).str.split('.').str[0]
if 'CZ_FIPS' in df_noaa.columns:
    df_noaa['CZ_FIPS'] = df_noaa['CZ_FIPS'].astype(str).str.split('.').str[0]


# --- Display Info and Head ---
print("\n--- NOAA DataFrame Info after initial cleaning ---")
df_noaa.info(verbose=True, show_counts=True)

print("\n--- NOAA DataFrame Head (focus on datetimes) ---")
print(df_noaa[['EVENT_ID', 'CZ_TIMEZONE', 'BEGIN_DT', 'BEGIN_DT_LOC', 'BEGIN_DT_UTC', 'END_DT_UTC']].head())

print("\n--- Data Types of Final Datetime Columns ---")
print(df_noaa[['BEGIN_DT_UTC', 'END_DT_UTC']].dtypes)

print("\nStep 1 (NOAA Load/Clean) Complete.")
# df_noaa now contains the loaded and initially cleaned NOAA data
# Key final columns: BEGIN_DT_UTC, END_DT_UTC (should be datetime64[ns, UTC])

Loading NOAA data from: data/NOAA_StormEvents/StormEvents_2014_2024.csv
NOAA data loaded successfully.
Performing initial cleaning and type conversions...
Parsing original datetime strings...
Mapping timezones...
Applying timezone localization (this may take time)...
Timezone localization applied.
Attempting to convert localized columns directly to UTC...
Direct UTC conversion attempted.
Converting numeric columns...
Converting FIPS codes to string...


/tmp/ipykernel_40165/3760516150.py:138: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_noaa[col].fillna(0, inplace=True)



--- NOAA DataFrame Info after initial cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691434 entries, 0 to 691433
Data columns (total 57 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   BEGIN_YEARMONTH     691434 non-null  int64              
 1   BEGIN_DAY           691434 non-null  int64              
 2   BEGIN_TIME          691434 non-null  int64              
 3   END_YEARMONTH       691434 non-null  int64              
 4   END_DAY             691434 non-null  int64              
 5   END_TIME            691434 non-null  int64              
 6   EPISODE_ID          691434 non-null  int64              
 7   EVENT_ID            691434 non-null  int64              
 8   STATE               691434 non-null  object             
 9   STATE_FIPS          691434 non-null  object             
 10  YEAR                691434 non-null  int64              
 11  MONTH_NAME          691434

In [12]:
df_noaa

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,BEGIN_DT,END_DT,BEGIN_DT_LOC,END_DT_LOC,BEGIN_DT_UTC,END_DT_UTC
0,201402,18,1000,201402,18,2000,83473,503953,NEW HAMPSHIRE,33,...,NaN,Low pressure developing south of Long Island a...,Eight to twelve inches of snow fell across eas...,CSV,2014-02-18 10:00:00,2014-02-18 20:00:00,2014-02-18 10:00:00-05:00,2014-02-18 20:00:00-05:00,2014-02-18 15:00:00+00:00,2014-02-19 01:00:00+00:00
1,201403,30,831,201403,30,931,83971,507163,MASSACHUSETTS,25,...,-71.3469,A stacked low pressure system passed south and...,Boston Road was closed near Brian Road due to ...,CSV,2014-03-30 08:31:00,2014-03-30 09:31:00,2014-03-30 08:31:00-04:00,2014-03-30 09:31:00-04:00,2014-03-30 12:31:00+00:00,2014-03-30 13:31:00+00:00
2,201404,27,2306,201404,27,2306,83517,506236,MISSOURI,29,...,-92.6600,A powerful storm system and a dry line produce...,NaN,CSV,2014-04-27 23:06:00,2014-04-27 23:06:00,2014-04-27 23:06:00-05:00,2014-04-27 23:06:00-05:00,2014-04-28 04:06:00+00:00,2014-04-28 04:06:00+00:00
3,201404,27,2303,201404,27,2303,83517,506237,MISSOURI,29,...,-92.6600,A powerful storm system and a dry line produce...,Several power poles snapped and trees blown down.,CSV,2014-04-27 23:03:00,2014-04-27 23:03:00,2014-04-27 23:03:00-05:00,2014-04-27 23:03:00-05:00,2014-04-28 04:03:00+00:00,2014-04-28 04:03:00+00:00
4,201402,15,1300,201402,15,2100,83132,501499,WASHINGTON,53,...,NaN,A strong cold front produced strong winds for ...,Two stations measured strong wind gusts in the...,CSV,2014-02-15 13:00:00,2014-02-15 21:00:00,2014-02-15 13:00:00-08:00,2014-02-15 21:00:00-08:00,2014-02-15 21:00:00+00:00,2014-02-16 05:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691429,202405,26,1148,202405,26,1148,192532,1188957,KENTUCKY,21,...,-84.7200,A strong storm system moved across the Ohio an...,A trained spotter estimated 60 mph wind gusts ...,CSV,2024-05-26 11:48:00,2024-05-26 11:48:00,2024-05-26 11:48:00-04:00,2024-05-26 11:48:00-04:00,2024-05-26 15:48:00+00:00,2024-05-26 15:48:00+00:00
691430,202405,22,1809,202405,22,1809,192530,1188234,INDIANA,18,...,-85.7364,A cold front moved into the Ohio Valley during...,A tree was down at Lovers Lane and Prewitt Lane.,CSV,2024-05-22 18:09:00,2024-05-22 18:09:00,2024-05-22 18:09:00-04:00,2024-05-22 18:09:00-04:00,2024-05-22 22:09:00+00:00,2024-05-22 22:09:00+00:00
691431,202405,22,1757,202405,22,1757,192530,1188232,INDIANA,18,...,-86.7247,A cold front moved into the Ohio Valley during...,A tree was reported down over Chestnut Grove R...,CSV,2024-05-22 17:57:00,2024-05-22 17:57:00,2024-05-22 17:57:00-04:00,2024-05-22 17:57:00-04:00,2024-05-22 21:57:00+00:00,2024-05-22 21:57:00+00:00
691432,202406,23,1745,202406,23,1750,191388,1192879,NEW HAMPSHIRE,33,...,-70.8400,A supercell thunderstorm developed across sout...,A supercell thunderstorm dropped hail the size...,CSV,2024-06-23 17:45:00,2024-06-23 17:50:00,2024-06-23 17:45:00-04:00,2024-06-23 17:50:00-04:00,2024-06-23 21:45:00+00:00,2024-06-23 21:50:00+00:00


In [11]:
df_noaa.to_csv("NOAA_timezone_cleaned.csv",index=False)

In [4]:
import pandas as pd
import numpy as np
import warnings
import re # Import regex module for damage parsing

# Assume df_noaa exists from Step 1

print("\nStarting Step 3: Feature Engineering on NOAA Data...")

# --- 1. Parse Damage Columns ---
print("Parsing DAMAGE_PROPERTY and DAMAGE_CROPS...")

def parse_damage_value(value):
    """Converts damage strings (e.g., '10.00K', '1.5M', '50B') to numeric."""
    if pd.isna(value) or value == '0.00K' or value == 0: # Handle NaNs and explicit zero
        return 0.0
    value_str = str(value).strip().upper()
    # Use regex to find number and optional multiplier (K, M, B)
    match = re.match(r'([\d\.]+)([KMB]?)', value_str)
    if match:
        number, multiplier = match.groups()
        number = float(number)
        if multiplier == 'K':
            return number * 1_000
        elif multiplier == 'M':
            return number * 1_000_000
        elif multiplier == 'B':
            return number * 1_000_000_000
        else: # No multiplier, assume direct value
            return number
    else:
        # warnings.warn(f"Could not parse damage value: {value}", UserWarning)
        return 0.0 # Return 0 if pattern doesn't match

# Apply the function
if 'DAMAGE_PROPERTY' in df_noaa.columns:
    df_noaa['DAMAGE_PROPERTY_NUMERIC'] = df_noaa['DAMAGE_PROPERTY'].apply(parse_damage_value)
else:
    print("Warning: DAMAGE_PROPERTY column not found.")
    df_noaa['DAMAGE_PROPERTY_NUMERIC'] = 0.0

if 'DAMAGE_CROPS' in df_noaa.columns:
    df_noaa['DAMAGE_CROPS_NUMERIC'] = df_noaa['DAMAGE_CROPS'].apply(parse_damage_value)
else:
    print("Warning: DAMAGE_CROPS column not found.")
    df_noaa['DAMAGE_CROPS_NUMERIC'] = 0.0

print("Damage columns parsed.")


# --- 2. Calculate Event Duration ---
print("Calculating event duration...")
if 'BEGIN_DT_UTC' in df_noaa.columns and 'END_DT_UTC' in df_noaa.columns:
    # Calculate duration in seconds, then convert to hours
    print("  Ensuring UTC columns are datetime type...")
    df_noaa['BEGIN_DT_UTC'] = pd.to_datetime(df_noaa['BEGIN_DT_UTC'])
    df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_UTC'])
    duration_seconds = (df_noaa['END_DT_UTC'] - df_noaa['BEGIN_DT_UTC']).dt.total_seconds()
    # Handle potential negative durations (if end < begin) by setting them to 0 or NaN
    duration_seconds[duration_seconds < 0] = 0
    df_noaa['EVENT_DURATION_HOURS'] = duration_seconds / 3600.0
    df_noaa['EVENT_DURATION_HOURS'].fillna(0, inplace=True) # Fill NaN durations with 0
else:
    print("Warning: BEGIN_DT_UTC or END_DT_UTC columns not found. Cannot calculate duration.")
    df_noaa['EVENT_DURATION_HOURS'] = 0.0
print("Event duration calculated.")


# --- 3. Handle Magnitude NaN ---
print("Handling MAGNITUDE NaNs...")
if 'MAGNITUDE' in df_noaa.columns:
    # Impute with 0 based on previous discussion (NaN often means not applicable)
    nan_before = df_noaa['MAGNITUDE'].isna().sum()
    df_noaa['MAGNITUDE'].fillna(0, inplace=True)
    print(f"  Filled {nan_before} NaNs in MAGNITUDE with 0.")
else:
    print("Warning: MAGNITUDE column not found.")
print("MAGNITUDE NaNs handled.")


# --- 4. Filter by CZ_TYPE (Recommended) ---
print("Filtering NOAA data for County-level events (CZ_TYPE == 'C')...")
if 'CZ_TYPE' in df_noaa.columns:
    rows_before = len(df_noaa)
    df_noaa = df_noaa[df_noaa['CZ_TYPE'] == 'C'].copy() # Filter and create a copy
    rows_after = len(df_noaa)
    print(f"  Retained {rows_after} rows out of {rows_before} (County-level events).")
else:
    print("Warning: CZ_TYPE column not found. Cannot filter by county-level events.")


# --- Display Info and Head ---
print("\n--- NOAA DataFrame Info after Step 3 Feature Engineering ---")
df_noaa.info(verbose=True, show_counts=True)

print("\n--- NOAA DataFrame Head (New Features) ---")
print(df_noaa[[
    'EVENT_ID', 'CZ_FIPS', 'BEGIN_DT_UTC', 'END_DT_UTC',
    'DAMAGE_PROPERTY', 'DAMAGE_PROPERTY_NUMERIC',
    'DAMAGE_CROPS', 'DAMAGE_CROPS_NUMERIC',
    'EVENT_DURATION_HOURS', 'MAGNITUDE'
]].head())

print("\nStep 3 (NOAA Feature Engineering) Complete.")
# df_noaa now has numeric damage columns, event duration, imputed magnitude,
# and is potentially filtered to only county-level events.


Starting Step 3: Feature Engineering on NOAA Data...
Parsing DAMAGE_PROPERTY and DAMAGE_CROPS...
Damage columns parsed.
Calculating event duration...
  Ensuring UTC columns are datetime type...
Event duration calculated.
Handling MAGNITUDE NaNs...
  Filled 331118 NaNs in MAGNITUDE with 0.
MAGNITUDE NaNs handled.
Filtering NOAA data for County-level events (CZ_TYPE == 'C')...


/tmp/ipykernel_40165/1085264281.py:60: SettingWithCopyWarning: modifications to a method of a datetimelike object are not supported and are discarded. Change values on the original.
  duration_seconds[duration_seconds < 0] = 0
/tmp/ipykernel_40165/1085264281.py:60: SettingWithCopyWarning: modifications to a method of a datetimelike object are not supported and are discarded. Change values on the original.
  duration_seconds[duration_seconds < 0] = 0
/tmp/ipykernel_40165/1085264281.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original objec

  Retained 397066 rows out of 691434 (County-level events).

--- NOAA DataFrame Info after Step 3 Feature Engineering ---
<class 'pandas.core.frame.DataFrame'>
Index: 397066 entries, 1 to 691433
Data columns (total 60 columns):
 #   Column                   Non-Null Count   Dtype              
---  ------                   --------------   -----              
 0   BEGIN_YEARMONTH          397066 non-null  int64              
 1   BEGIN_DAY                397066 non-null  int64              
 2   BEGIN_TIME               397066 non-null  int64              
 3   END_YEARMONTH            397066 non-null  int64              
 4   END_DAY                  397066 non-null  int64              
 5   END_TIME                 397066 non-null  int64              
 6   EPISODE_ID               397066 non-null  int64              
 7   EVENT_ID                 397066 non-null  int64              
 8   STATE                    397066 non-null  object             
 9   STATE_FIPS               39706

In [5]:
import pandas as pd
import glob # To find the files
import os

# Define the directory containing the Eaglei data
eaglei_dir = 'data/eaglei_data/'

# Find all eaglei outage CSV files
eaglei_files = glob.glob(os.path.join(eaglei_dir, 'eaglei_outages_*.csv'))

if not eaglei_files:
    print(f"Error: No 'eaglei_outages_*.csv' files found in {eaglei_dir}")
    exit()
else:
    print(f"Found {len(eaglei_files)} Eaglei outage files:")
    eaglei_files.sort()
    for f in eaglei_files[:3]: print(f"  - {os.path.basename(f)}")
    if len(eaglei_files) > 3: print("  - ... and others")


# --- Load and Concatenate Data ---
print("\nLoading and concatenating Eaglei data...")
list_of_dfs = []
for f in eaglei_files:
    try:
        df_temp = pd.read_csv(f)
        list_of_dfs.append(df_temp)
        print(f"  Loaded {os.path.basename(f)}")
    except FileNotFoundError:
        print(f"  Warning: File {f} not found during loading loop.")
    except Exception as e:
        print(f"  Error loading {os.path.basename(f)}: {e}")

if not list_of_dfs:
    print("Error: Failed to load any Eaglei data.")
    exit()

df_eaglei = pd.concat(list_of_dfs, ignore_index=True)
print("Eaglei data concatenated successfully.")


# --- Initial Cleaning and Type Conversions ---
print("\nPerforming initial cleaning and type conversions on Eaglei data...")

# Datetime Conversion (Knowing Input is UTC)
eaglei_datetime_format = '%Y-%m-%d %H:%M:%S'
print(f"Converting 'run_start_time' (known UTC) using format: {eaglei_datetime_format}")

# Parse the datetime string. Initially, this creates a naive datetime object
# because the string format itself doesn't contain timezone info.
df_eaglei['run_start_time_parsed'] = pd.to_datetime(df_eaglei['run_start_time'], format=eaglei_datetime_format, errors='coerce')

# Check for parsing errors
naive_nat_count = df_eaglei['run_start_time_parsed'].isna().sum()
if naive_nat_count > 0:
    print(f"Warning: Found {naive_nat_count} NaT values after parsing 'run_start_time'.")
    # Optional: drop rows with invalid dates if needed
    # df_eaglei.dropna(subset=['run_start_time_parsed'], inplace=True)

# *** Assign UTC timezone ***
# Since we know the original times represent UTC moments, we *localize* the naive
# datetime objects to UTC. This tells pandas these times are UTC.
print("Assigning UTC timezone to parsed 'run_start_time'...")
try:
    df_eaglei['EAGLEI_DT_UTC'] = df_eaglei['run_start_time_parsed'].dt.tz_localize('UTC')
except AttributeError as e:
     # Fallback in case parsing resulted in 'object' dtype
     print(f"Error: Failed using .dt accessor. Attempting direct conversion specifying UTC...")
     df_eaglei['EAGLEI_DT_UTC'] = pd.to_datetime(df_eaglei['run_start_time_parsed'], errors='coerce', utc=True)
except Exception as e:
    print(f"Error during UTC assignment: {e}")
    df_eaglei['EAGLEI_DT_UTC'] = pd.NaT

utc_nat_count = df_eaglei['EAGLEI_DT_UTC'].isna().sum()
if utc_nat_count > naive_nat_count:
     print(f"Warning: {utc_nat_count - naive_nat_count} additional NaTs introduced during UTC assignment.")

# Can drop the intermediate parsed column now
df_eaglei.drop(columns=['run_start_time_parsed'], inplace=True, errors='ignore')


# FIPS Code Formatting
print("Formatting 'fips_code'...")
if 'fips_code' in df_eaglei.columns:
    df_eaglei['fips_code'] = df_eaglei['fips_code'].astype(str)
    df_eaglei['fips_code'] = df_eaglei['fips_code'].str.zfill(5)
else:
    print("Warning: 'fips_code' column not found in Eaglei data!")


# Customers Out Type Check
print("Checking 'customers_out' type...")
if 'customers_out' in df_eaglei.columns:
    df_eaglei['customers_out'] = pd.to_numeric(df_eaglei['customers_out'], errors='coerce')
    customers_out_nan_before = df_eaglei['customers_out'].isna().sum()
    df_eaglei['customers_out'].fillna(0, inplace=True)
    customers_out_nan_after = df_eaglei['customers_out'].isna().sum()
    if customers_out_nan_before > customers_out_nan_after:
        print(f"  Filled {customers_out_nan_before - customers_out_nan_after} NaN values in 'customers_out' with 0.")
    # Optionally convert to integer if appropriate
    # df_eaglei['customers_out'] = df_eaglei['customers_out'].astype(int)
else:
    print("Warning: 'customers_out' column not found in Eaglei data!")


# --- Display Info and Head ---
print("\n--- Eaglei DataFrame Info after initial cleaning ---")
df_eaglei.info(verbose=True, show_counts=True)

print("\n--- Eaglei DataFrame Head (Cleaned) ---")
print(df_eaglei[['fips_code', 'run_start_time', 'EAGLEI_DT_UTC', 'customers_out']].head())

print("\n--- Data Types of Final Eaglei Columns ---")
print(df_eaglei[['fips_code', 'EAGLEI_DT_UTC', 'customers_out']].dtypes)

print("\nStep 2 (Eaglei Load/Clean) Complete - Processed as UTC.")
# df_eaglei now contains the loaded and cleaned Eaglei data
# Key columns: fips_code (string, padded), EAGLEI_DT_UTC (datetime64[ns, UTC]), customers_out (numeric)


Found 10 Eaglei outage files:
  - eaglei_outages_2014.csv
  - eaglei_outages_2015.csv
  - eaglei_outages_2016.csv
  - ... and others

Loading and concatenating Eaglei data...
  Loaded eaglei_outages_2014.csv
  Loaded eaglei_outages_2015.csv
  Loaded eaglei_outages_2016.csv
  Loaded eaglei_outages_2017.csv
  Loaded eaglei_outages_2018.csv
  Loaded eaglei_outages_2019.csv
  Loaded eaglei_outages_2020.csv
  Loaded eaglei_outages_2021.csv
  Loaded eaglei_outages_2022.csv
  Loaded eaglei_outages_2023.csv
Eaglei data concatenated successfully.

Performing initial cleaning and type conversions on Eaglei data...
Converting 'run_start_time' (known UTC) using format: %Y-%m-%d %H:%M:%S
Assigning UTC timezone to parsed 'run_start_time'...
Formatting 'fips_code'...
Checking 'customers_out' type...


/tmp/ipykernel_40165/1915395210.py:96: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_eaglei['customers_out'].fillna(0, inplace=True)


  Filled 6763505 NaN values in 'customers_out' with 0.

--- Eaglei DataFrame Info after initial cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191133068 entries, 0 to 191133067
Data columns (total 6 columns):
 #   Column          Non-Null Count      Dtype              
---  ------          --------------      -----              
 0   fips_code       191133068 non-null  object             
 1   county          191133068 non-null  object             
 2   state           191133068 non-null  object             
 3   customers_out   191133068 non-null  float64            
 4   run_start_time  191133068 non-null  object             
 5   EAGLEI_DT_UTC   191133068 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 8.5+ GB

--- Eaglei DataFrame Head (Cleaned) ---
  fips_code       run_start_time             EAGLEI_DT_UTC  customers_out
0     01037  2014-11-01 04:00:00 2014-11-01 04:00:00+00:00           12.0
1     01051  2014-11-0

In [14]:
df_eaglei


,fips_code,county,state,customers_out,run_start_time,run_start_time_naive,EAGLEI_DT_UTC
0,01037,Coosa,Alabama,12.0,2014-11-01 04:00:00,2014-11-01 04:00:00,2014-11-01 04:00:00+00:00
1,01051,Elmore,Alabama,7.0,2014-11-01 04:00:00,2014-11-01 04:00:00,2014-11-01 04:00:00+00:00
2,01109,Pike,Alabama,1.0,2014-11-01 04:00:00,2014-11-01 04:00:00,2014-11-01 04:00:00+00:00
3,01121,Talladega,Alabama,31.0,2014-11-01 04:00:00,2014-11-01 04:00:00,2014-11-01 04:00:00+00:00
4,04017,Navajo,Arizona,1.0,2014-11-01 04:00:00,2014-11-01 04:00:00,2014-11-01 04:00:00+00:00
...,...,...,...,...,...,...,...
191133063,55095,Polk,Wisconsin,0.0,2023-12-31 23:45:00,2023-12-31 23:45:00,2023-12-31 23:45:00+00:00
191133064,55105,Rock,Wisconsin,1.0,2023-12-31 23:45:00,2023-12-31 23:45:00,2023-12-31 23:45:00+00:00
191133065,55109,St. Croix,Wisconsin,0.0,2023-12-31 23:45:00,2023-12-31 23:45:00,2023-12-31 23:45:00+00:00
191133066,55129,Washburn,Wisconsin,0.0,2023-12-31 23:45:00,2023-12-31 23:45:00,2023-12-31 23:45:00+00:00


In [7]:
df_eaglei.to_pickle("eaglei_2014_2024.pkl")

In [18]:
df_eaglei = pd.read_csv("eaglei_2014_2024.csv")
unq = df_noaa['CZ_NAME'].unique()
unq1 = df_eaglei['county'].unique()
print(unq1)

['Coosa' 'Elmore' 'Pike' ... 'Taos' 'Toole' 'Prentiss']


In [19]:
# Get unique values (with NaN handling)
noaa_counties = set(df_noaa['CZ_NAME'].dropna().str.upper().unique())
eaglei_counties = set(df_eaglei['county'].dropna().str.upper().unique())

# Check equality
if noaa_counties == eaglei_counties:
    print("Both datasets contain EXACTLY the same county names")
else:
    # Calculate differences
    only_in_noaa = noaa_counties - eaglei_counties
    only_in_eaglei = eaglei_counties - noaa_counties
    
    print(f"Difference count: {len(only_in_noaa) + len(only_in_eaglei)}")
    print(f"Unique to NOAA: {len(only_in_noaa)}")
    print(f"Unique to EagleI: {len(only_in_eaglei)}")
    
    # Optional: Show examples of mismatches
    print("\nSample unique to NOAA:", list(only_in_noaa)[:3])
    print("Sample unique to EagleI:", list(only_in_eaglei)[:3])


Difference count: 220
Unique to NOAA: 176
Unique to EagleI: 44

Sample unique to NOAA: ['ALASKA PENINSULA', 'VIEQUES', 'QUEBRADILLAS']
Sample unique to EagleI: ['CARSON CITY', 'KALAWAO', 'COLONIAL HEIGHTS']


In [8]:
df_noaa.to_pickle('noaa_feature.pkl')

In [1]:
import pandas as pd
df_noaa = pd.read_csv("noaa_minimal_feature_eng.csv")
df_eaglei = pd.read_csv("eaglei_2014_2024.csv")

In [9]:
df_eaglei['fips_code'].head(2)

0    01037
1    01051
Name: fips_code, dtype: object

In [5]:
# --- Check for non-numeric values in original STATE_FIPS and CZ_FIPS ---
print("\nChecking for non-numeric values in STATE_FIPS and CZ_FIPS...")

# Check STATE_FIPS
non_numeric_state = df_noaa[~df_noaa['STATE_FIPS'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(f"Rows with non-numeric STATE_FIPS: {len(non_numeric_state)}")
if not non_numeric_state.empty:
    print("Examples of invalid STATE_FIPS:")
    print(non_numeric_state[['STATE_FIPS']].head())

# Check CZ_FIPS
non_numeric_cz = df_noaa[~df_noaa['CZ_FIPS'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(f"\nRows with non-numeric CZ_FIPS: {len(non_numeric_cz)}")
if not non_numeric_cz.empty:
    print("Examples of invalid CZ_FIPS:")
    print(non_numeric_cz[['CZ_FIPS']].head())
non_numeric_fp = df_eaglei[~df_eaglei['fips_code'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(f"\nRows with non-numeric fp_FIPS: {len(non_numeric_fp)}")



Checking for non-numeric values in STATE_FIPS and CZ_FIPS...
Rows with non-numeric STATE_FIPS: 0

Rows with non-numeric CZ_FIPS: 0

Rows with non-numeric fp_FIPS: 0


In [1]:
import pandas as pd
import numpy as np
import warnings

# Assume df_noaa exists from Step 3
# Assume df_eaglei exists from Step 2
df_noaa = pd.read_pickle('noaa_feature.pkl')
df_eaglei = pd.read_pickle('eaglei_2014_2024.pkl')

print("\nStarting Step 4: Merging NOAA and Eaglei Data...")

# --- 1. Create full 5-digit FIPS code in df_noaa ---
print("Creating 5-digit FIPS code in NOAA data...")
if 'STATE_FIPS' in df_noaa.columns and 'CZ_FIPS' in df_noaa.columns:
    # Ensure source columns are strings and handle potential floats/NaNs gracefully first
    df_noaa['STATE_FIPS_STR'] = df_noaa['STATE_FIPS'].astype(str).str.split('.').str[0]
    df_noaa['CZ_FIPS_STR'] = df_noaa['CZ_FIPS'].astype(str).str.split('.').str[0]

    # Pad STATE_FIPS to 2 digits, CZ_FIPS to 3 digits
    state_fips_padded = df_noaa['STATE_FIPS_STR'].str.zfill(2)
    cz_fips_padded = df_noaa['CZ_FIPS_STR'].str.zfill(3)

    # Concatenate, but only if both parts are valid (not NaN after conversion)
    # Create mask for valid rows
    valid_fips_mask = state_fips_padded.notna() & cz_fips_padded.notna()
    df_noaa['full_fips_code'] = np.nan # Initialize with NaN
    df_noaa.loc[valid_fips_mask, 'full_fips_code'] = state_fips_padded[valid_fips_mask] + cz_fips_padded[valid_fips_mask]

    # Drop intermediate columns
    df_noaa.drop(columns=['STATE_FIPS_STR', 'CZ_FIPS_STR'], inplace=True)

    # Check how many NaNs were created
    fips_nan_count = df_noaa['full_fips_code'].isna().sum()
    if fips_nan_count > 0:
        print(f"  Warning: Created {fips_nan_count} NaN values in 'full_fips_code' due to missing STATE or CZ FIPS.")
        # Optional: Drop rows with missing FIPS before merging if needed
        # df_noaa.dropna(subset=['full_fips_code'], inplace=True)

    print("  'full_fips_code' created.")
    # Verify one FIPS code
    print(f"  Example NOAA full_fips_code: {df_noaa['full_fips_code'].iloc[0] if not df_noaa.empty else 'N/A'}")

else:
    print("Error: STATE_FIPS or CZ_FIPS missing from df_noaa. Cannot create full FIPS code.")
    # Handle error - cannot proceed with merge
    exit()

# Verify Eaglei FIPS code formatting (should be done already)
print(f"  Example Eaglei fips_code: {df_eaglei['fips_code'].iloc[0] if not df_eaglei.empty else 'N/A'}")

df_noaa.to_pickle("noaa_fips.pkl")


Starting Step 4: Merging NOAA and Eaglei Data...
Creating 5-digit FIPS code in NOAA data...


/tmp/ipykernel_35761/561022568.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['25017' '29067' '29067' ... '18123' '33015' '13051']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_noaa.loc[valid_fips_mask, 'full_fips_code'] = state_fips_padded[valid_fips_mask] + cz_fips_padded[valid_fips_mask]


  'full_fips_code' created.
  Example NOAA full_fips_code: 25017
  Example Eaglei fips_code: 01037


In [2]:

# --- 2. Prepare df_eaglei Key (already done) ---
# Ensure required columns exist
if not all(col in df_eaglei.columns for col in ['fips_code', 'EAGLEI_DT_UTC', 'customers_out']):
     print("Error: Required columns missing from df_eaglei.")
     exit()
# Select only necessary columns from Eaglei to save memory during merge
df_eaglei_subset = df_eaglei[['fips_code', 'EAGLEI_DT_UTC', 'customers_out']].copy()
print(f"Prepared Eaglei subset with {len(df_eaglei_subset)} rows.")

#Handled mismatch in data_type
df_eaglei_subset['EAGLEI_DT_UTC'] = pd.to_datetime(df_eaglei_subset['EAGLEI_DT_UTC'])
df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_UTC'])
# And similarly for df_noaa['END_DT_UTC']
# df_eaglei_subset['fips_code'] = df_eaglei_subset['fips_code'].astype(str).str.zfill(5)
# This line below specifically targets the NaNs in END_DT_UTC
df_noaa.dropna(subset=['END_DT_UTC', 'full_fips_code'], inplace=True)




# --- 4. Define Target & Tolerance ---
# Let's look for outages within 6 hours AFTER a storm ends
merge_tolerance = pd.Timedelta('6h')
print(f"Merge tolerance set to: {merge_tolerance}")
print(df_noaa['END_DT_UTC'].dtype)
print(df_noaa['full_fips_code'].dtype)
print(df_eaglei_subset['EAGLEI_DT_UTC'].dtype)
print(df_eaglei_subset['fips_code'].dtype)

# --- CRITICAL FIX: Remove rows with nulls in ANY key column ---
# ...
# --- 3. Sort DataFrames ---
print("Sorting DataFrames (this might take time)...")
# Sort NOAA by the new full FIPS and storm END time
df_noaa.sort_values(by=['full_fips_code', 'END_DT_UTC'], inplace=True)
# Sort Eaglei subset by FIPS and outage time
df_eaglei_subset.sort_values(by=['fips_code', 'EAGLEI_DT_UTC'], inplace=True)
print("DataFrames sorted.")
df_noaa

Prepared Eaglei subset with 191133068 rows.
Merge tolerance set to: 0 days 06:00:00
datetime64[ns, UTC]
object
datetime64[ns, UTC]
object
Sorting DataFrames (this might take time)...
DataFrames sorted.


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,BEGIN_DT,END_DT,BEGIN_DT_LOC,END_DT_LOC,BEGIN_DT_UTC,END_DT_UTC,DAMAGE_PROPERTY_NUMERIC,DAMAGE_CROPS_NUMERIC,EVENT_DURATION_HOURS,full_fips_code
44383,201401,11,615,201401,11,615,81761,494246,ALABAMA,1,...,2014-01-11 06:15:00,2014-01-11 06:15:00,2014-01-11 06:15:00-06:00,2014-01-11 06:15:00-06:00,2014-01-11 12:15:00+00:00,2014-01-11 12:15:00+00:00,0.0,0.0,0.000000,01001
44384,201401,11,620,201401,11,620,81761,494247,ALABAMA,1,...,2014-01-11 06:20:00,2014-01-11 06:20:00,2014-01-11 06:20:00-06:00,2014-01-11 06:20:00-06:00,2014-01-11 12:20:00+00:00,2014-01-11 12:20:00+00:00,0.0,0.0,0.000000,01001
17465,201404,7,715,201404,7,1335,85674,521252,ALABAMA,1,...,2014-04-07 07:15:00,2014-04-07 13:35:00,2014-04-07 07:15:00-05:00,2014-04-07 13:35:00-05:00,2014-04-07 12:15:00+00:00,2014-04-07 18:35:00+00:00,0.0,0.0,6.333333,01001
18130,201404,7,900,201404,8,1400,85674,524756,ALABAMA,1,...,2014-04-07 09:00:00,2014-04-08 14:00:00,2014-04-07 09:00:00-05:00,2014-04-08 14:00:00-05:00,2014-04-07 14:00:00+00:00,2014-04-08 19:00:00+00:00,0.0,0.0,29.000000,01001
29615,201404,30,230,201404,30,230,83782,523313,ALABAMA,1,...,2014-04-30 02:30:00,2014-04-30 02:30:00,2014-04-30 02:30:00-05:00,2014-04-30 02:30:00-05:00,2014-04-30 07:30:00+00:00,2014-04-30 07:30:00+00:00,0.0,0.0,0.000000,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635761,202405,7,1711,202405,7,2015,192139,1184437,PUERTO RICO,99,...,2024-05-07 17:11:00,2024-05-07 20:15:00,2024-05-07 17:11:00-04:00,2024-05-07 20:15:00-04:00,2024-05-07 21:11:00+00:00,2024-05-08 00:15:00+00:00,3000.0,0.0,3.066667,99153
630167,202405,8,1410,202405,8,1800,192139,1184610,PUERTO RICO,99,...,2024-05-08 14:10:00,2024-05-08 18:00:00,2024-05-08 14:10:00-04:00,2024-05-08 18:00:00-04:00,2024-05-08 18:10:00+00:00,2024-05-08 22:00:00+00:00,0.0,0.0,3.833333,99153
630182,202405,8,1450,202405,8,1800,192139,1184611,PUERTO RICO,99,...,2024-05-08 14:50:00,2024-05-08 18:00:00,2024-05-08 14:50:00-04:00,2024-05-08 18:00:00-04:00,2024-05-08 18:50:00+00:00,2024-05-08 22:00:00+00:00,1000.0,0.0,3.166667,99153
630446,202405,8,1406,202405,8,1800,192139,1184609,PUERTO RICO,99,...,2024-05-08 14:06:00,2024-05-08 18:00:00,2024-05-08 14:06:00-04:00,2024-05-08 18:00:00-04:00,2024-05-08 18:06:00+00:00,2024-05-08 22:00:00+00:00,5000.0,0.0,3.900000,99153


In [7]:
import pandas as pd
import numpy as np
import warnings

# Assume df_noaa exists from Step 3
# Assume df_eaglei exists from Step 2
df_noaa = pd.read_pickle("noaa_fips.pkl")
print("\nStarting Step 4: Merging NOAA and Eaglei Data...")

# --- 1. Create full 5-digit FIPS code in df_noaa ---
# print("Creating 5-digit FIPS code in NOAA data...")
# if 'STATE_FIPS' in df_noaa.columns and 'CZ_FIPS' in df_noaa.columns:
#     # Ensure source columns are strings and handle potential floats/NaNs gracefully first
#     df_noaa['STATE_FIPS_STR'] = df_noaa['STATE_FIPS'].astype(str).str.split('.').str[0]
#     df_noaa['CZ_FIPS_STR'] = df_noaa['CZ_FIPS'].astype(str).str.split('.').str[0]

#     # Pad STATE_FIPS to 2 digits, CZ_FIPS to 3 digits
#     state_fips_padded = df_noaa['STATE_FIPS_STR'].str.zfill(2)
#     cz_fips_padded = df_noaa['CZ_FIPS_STR'].str.zfill(3)

#     # Concatenate, but only if both parts are valid (not NaN after conversion)
#     # Create mask for valid rows
#     valid_fips_mask = state_fips_padded.notna() & cz_fips_padded.notna()
#     df_noaa['full_fips_code'] = np.nan # Initialize with NaN
#     df_noaa.loc[valid_fips_mask, 'full_fips_code'] = state_fips_padded[valid_fips_mask] + cz_fips_padded[valid_fips_mask]

#     # Drop intermediate columns
#     df_noaa.drop(columns=['STATE_FIPS_STR', 'CZ_FIPS_STR'], inplace=True)

#     # Check how many NaNs were created
#     fips_nan_count = df_noaa['full_fips_code'].isna().sum()
#     if fips_nan_count > 0:
#         print(f"  Warning: Created {fips_nan_count} NaN values in 'full_fips_code' due to missing STATE or CZ FIPS.")
#         # Optional: Drop rows with missing FIPS before merging if needed
#         # df_noaa.dropna(subset=['full_fips_code'], inplace=True)

#     print("  'full_fips_code' created.")
#     # Verify one FIPS code
#     print(f"  Example NOAA full_fips_code: {df_noaa['full_fips_code'].iloc[0] if not df_noaa.empty else 'N/A'}")

# else:
#     print("Error: STATE_FIPS or CZ_FIPS missing from df_noaa. Cannot create full FIPS code.")
#     # Handle error - cannot proceed with merge
#     exit()

# Verify Eaglei FIPS code formatting (should be done already)
print(f"  Example Eaglei fips_code: {df_eaglei['fips_code'].iloc[0] if not df_eaglei.empty else 'N/A'}")


# --- 2. Prepare df_eaglei Key (already done) ---
# Ensure required columns exist
if not all(col in df_eaglei.columns for col in ['fips_code', 'EAGLEI_DT_UTC', 'customers_out']):
     print("Error: Required columns missing from df_eaglei.")
     exit()
# Select only necessary columns from Eaglei to save memory during merge
df_eaglei_subset = df_eaglei[['fips_code', 'EAGLEI_DT_UTC', 'customers_out']].copy()
print(f"Prepared Eaglei subset with {len(df_eaglei_subset)} rows.")

#Handled mismatch in data_type
df_eaglei_subset['EAGLEI_DT_UTC'] = pd.to_datetime(df_eaglei_subset['EAGLEI_DT_UTC'])
df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_UTC'])
# And similarly for df_noaa['END_DT_UTC']
df_eaglei_subset['fips_code'] = df_eaglei_subset['fips_code'].astype(str).str.zfill(5)
# This line below specifically targets the NaNs in END_DT_UTC
df_noaa.dropna(subset=['END_DT_UTC', 'full_fips_code'], inplace=True)


df_noaa.reset_index(drop=True, inplace=True)
df_eaglei_subset.reset_index(drop=True, inplace=True)

# --- 4. Define Target & Tolerance ---
# Let's look for outages within 6 hours AFTER a storm ends
merge_tolerance = pd.Timedelta('6h')
print(f"Merge tolerance set to: {merge_tolerance}")
print(df_noaa['END_DT_UTC'].dtype)
print(df_noaa['full_fips_code'].dtype)
print(df_eaglei_subset['EAGLEI_DT_UTC'].dtype)
print(df_eaglei_subset['fips_code'].dtype)

# --- CRITICAL FIX: Remove rows with nulls in ANY key column ---
# ...

# Convert to integers (assuming no leading zeros are needed)
df_noaa['full_fips_code_int'] = df_noaa['full_fips_code'].astype(int)
df_eaglei_subset['fips_code_int'] = df_eaglei_subset['fips_code'].astype(int)

# --- 3. Sort DataFrames ---
print("Sorting DataFrames (this might take time)...")
# Sort NOAA by the new full FIPS and storm END time
df_noaa = df_noaa.sort_values(by=['full_fips_code', 'END_DT_UTC'])
# Sort Eaglei subset by FIPS and outage time
df_eaglei_subset = df_eaglei_subset.sort_values(by=['fips_code', 'EAGLEI_DT_UTC'])
print("DataFrames sorted.")

print(f"  Example NOAA full_fips_code: {df_noaa['full_fips_code'].iloc[0] if not df_noaa.empty else 'N/A'}")
print(f"  Example Eaglei fips_code: {df_eaglei['fips_code'].iloc[0] if not df_eaglei.empty else 'N/A'}")

print("\n--- Verifying df_noaa is properly sorted for merge_asof ---")

# Check if full_fips_code is monotonically increasing
fips_sorted = df_noaa['full_fips_code'].is_monotonic_increasing
print(f"1. full_fips_code is monotonically increasing: {fips_sorted}")

if not fips_sorted:
    # Find where the sorting breaks
    diff = df_noaa['full_fips_code'].iloc[1:] < df_noaa['full_fips_code'].iloc[:-1].values
    if diff.any():
        first_violation = diff.idxmax()
        print(f"  Sorting violation found at index {first_violation}:")
        print(f"    {first_violation-1}: {df_noaa['full_fips_code'].iloc[first_violation-1]}")
        print(f"    {first_violation}: {df_noaa['full_fips_code'].iloc[first_violation]}")

# Check if END_DT_UTC is sorted within each FIPS group
group_checks = df_noaa.groupby('full_fips_code')['END_DT_UTC'].apply(lambda x: x.is_monotonic_increasing)
all_groups_sorted = group_checks.all()
print(f"2. END_DT_UTC is sorted within each full_fips_code group: {all_groups_sorted}")

if not all_groups_sorted:
    bad_groups = group_checks[~group_checks].index.tolist()
    print(f"  Found {len(bad_groups)} groups with sorting issues:")
    for i, fips in enumerate(bad_groups[:3]):  # Show first 3 problematic groups
        group = df_noaa[df_noaa['full_fips_code'] == fips]
        print(f"  - FIPS {fips} (size {len(group)}):")
        # Find where the sorting breaks within this group
        diff = group['END_DT_UTC'].iloc[1:] < group['END_DT_UTC'].iloc[:-1].values
        if diff.any():
            first_violation = diff.idxmax()
            print(f"    Out-of-order at index {first_violation}:")
            print(f"      {first_violation-1}: {group.loc[first_violation-1, 'END_DT_UTC']}")
            print(f"      {first_violation}: {group.loc[first_violation, 'END_DT_UTC']}")

# Only proceed with merge if both checks pass
if fips_sorted and all_groups_sorted:
    print("✓ df_noaa is properly sorted for merge_asof")
else:
    print("✗ df_noaa is NOT properly sorted for merge_asof - must fix before merging")
    # Consider adding: exit() or df_noaa = df_noaa.sort_values(...) to auto-fix

print("Performing merge_asof (this is memory intensive and may take significant time)...")
try:
    # Merge df_noaa (left) with df_eaglei_subset (right)
    # For each storm in df_noaa, find the Eaglei record matching FIPS
    # where Eaglei time is >= storm end time, within the tolerance.
    merged_df = pd.merge_asof(
        df_noaa,                     # Left DataFrame
        df_eaglei_subset,            # Right DataFrame (subset)
        left_on='END_DT_UTC',        # Time column in left df (storm end)
        right_on='EAGLEI_DT_UTC',    # Time column in right df (outage time)
        left_by='full_fips_code',    # Key column in left df (county FIPS)
        right_by='fips_code',        # Key column in right df (county FIPS)
        direction='forward',         # Find Eaglei time >= left time
        tolerance=merge_tolerance    # Look ahead up to 6 hours
    )
    print("merge_asof complete.")

    # Rename columns coming from Eaglei for clarity, handling potential conflicts
    merged_df.rename(columns={
        'EAGLEI_DT_UTC': 'MATCHED_OUTAGE_DT_UTC',
        'customers_out': 'MATCHED_CUSTOMERS_OUT'
        # Note: 'fips_code' column from df_eaglei_subset will also be present
    }, inplace=True)

    # Check results
    print(f"Merged DataFrame length: {len(merged_df)}")
    if len(merged_df) != len(df_noaa):
         print("Warning: Length of merged df differs from original NOAA df. This shouldn't happen with merge_asof.")

    # See how many storms found a matching outage record within the window
    matches_found = merged_df['MATCHED_OUTAGE_DT_UTC'].notna().sum()
    print(f"Found matching outage records for {matches_found} out of {len(merged_df)} storm events within the {merge_tolerance} window.")

except MemoryError:
    print("!!! MEMORY ERROR during merge_asof !!!")
    print("The DataFrames are too large for a direct merge in available memory.")
    print("Next steps would require chunking, Dask, or more memory.")
    merged_df = None # Indicate failure
except Exception as e:
    print(f"An unexpected error occurred during merge_asof: {e}")
    merged_df = None # Indicate failure


# --- Display Info and Head of Merged Data (if successful) ---
if merged_df is not None:
    print("\n--- Merged DataFrame Info ---")
    merged_df.info(verbose=True, show_counts=True)

    print("\n--- Merged DataFrame Head ---")
    print(merged_df[[
        'EVENT_ID', 'full_fips_code', 'END_DT_UTC',
        'MATCHED_OUTAGE_DT_UTC', 'MATCHED_CUSTOMERS_OUT'
        # Add other relevant NOAA columns to view
    ]].head())

    print("\nStep 4 (Merge) Complete.")
else:
    print("\nStep 4 (Merge) FAILED due to errors.")

# 'merged_df' now holds the result IF the merge was successful.
# It has one row per original NOAA storm event.
# Rows where no outage was found within the window will have NaT/NaN
# in MATCHED_OUTAGE_DT_UTC and MATCHED_CUSTOMERS_OUT.



Starting Step 4: Merging NOAA and Eaglei Data...
  Example Eaglei fips_code: 01037
Prepared Eaglei subset with 191133068 rows.
Merge tolerance set to: 0 days 06:00:00
datetime64[ns, UTC]
object
datetime64[ns, UTC]
object
Sorting DataFrames (this might take time)...
DataFrames sorted.
  Example NOAA full_fips_code: 01001
  Example Eaglei fips_code: 01037

--- Verifying df_noaa is properly sorted for merge_asof ---
1. full_fips_code is monotonically increasing: True
2. END_DT_UTC is sorted within each full_fips_code group: True
✓ df_noaa is properly sorted for merge_asof
Performing merge_asof (this is memory intensive and may take significant time)...
An unexpected error occurred during merge_asof: left keys must be sorted

Step 4 (Merge) FAILED due to errors.


In [2]:
import pandas as pd
df_noaa = pd.read_pickle('noaa_fips.pkl')
df_eaglei = pd.read_pickle('eaglei_2014_2024.pkl')

# --- 2. Prepare df_eaglei Key (already done) ---
# Ensure required columns exist
if not all(col in df_eaglei.columns for col in ['fips_code', 'EAGLEI_DT_UTC', 'customers_out']):
     print("Error: Required columns missing from df_eaglei.")
     exit()
# Select only necessary columns from Eaglei to save memory during merge
df_eaglei_subset = df_eaglei[['fips_code', 'EAGLEI_DT_UTC', 'customers_out']].copy()
print(f"Prepared Eaglei subset with {len(df_eaglei_subset)} rows.")

#Handled mismatch in data_type
df_eaglei_subset['EAGLEI_DT_UTC'] = pd.to_datetime(df_eaglei_subset['EAGLEI_DT_UTC'])
df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_UTC'])
# And similarly for df_noaa['END_DT_UTC']
# df_eaglei_subset['fips_code'] = df_eaglei_subset['fips_code'].astype(str).str.zfill(5)
# This line below specifically targets the NaNs in END_DT_UTC
df_noaa.dropna(subset=['END_DT_UTC', 'full_fips_code'], inplace=True)




# --- 4. Define Target & Tolerance ---
# Let's look for outages within 6 hours AFTER a storm ends
merge_tolerance = pd.Timedelta('6h')
print(f"Merge tolerance set to: {merge_tolerance}")
print(df_noaa['END_DT_UTC'].dtype)
print(df_noaa['full_fips_code'].dtype)
print(df_eaglei_subset['EAGLEI_DT_UTC'].dtype)
print(df_eaglei_subset['fips_code'].dtype)

# --- CRITICAL FIX: Remove rows with nulls in ANY key column ---
# ...
# --- 3. Sort DataFrames ---
print("Sorting DataFrames (this might take time)...")
# Sort NOAA by the new full FIPS and storm END time
df_noaa.sort_values(by=['full_fips_code', 'END_DT_UTC'], inplace=True)
# Sort Eaglei subset by FIPS and outage time
df_eaglei_subset.sort_values(by=['fips_code', 'EAGLEI_DT_UTC'], inplace=True)
print("DataFrames sorted.")

Prepared Eaglei subset with 191133068 rows.
Merge tolerance set to: 0 days 06:00:00
datetime64[ns, UTC]
object
datetime64[ns, UTC]
object
Sorting DataFrames (this might take time)...
DataFrames sorted.


In [3]:
# First, standardize all FIPS codes (including the "bad" ones)
def standardize_fips(fips):
    try:
        fips = str(int(float(fips))).zfill(5)
        return fips if fips[:2] in [f"{i:02d}" for i in range(1,57)] else None
    except:
        return None

# Apply standardization
df_noaa['full_fips_code'] = df_noaa['full_fips_code'].apply(standardize_fips)
df_eaglei_subset['fips_code'] = df_eaglei_subset['fips_code'].apply(standardize_fips)

# Then filter to only valid FIPS codes
valid_state_codes = [f"{i:02d}" for i in range(1,57)]  # 01-56
df_noaa = df_noaa[df_noaa['full_fips_code'].str[:2].isin(valid_state_codes)]
df_eaglei_subset = df_eaglei_subset[df_eaglei_subset['fips_code'].str[:2].isin(valid_state_codes)]

# Verify
print(f"NOAA clean count: {len(df_noaa)}")
print(f"EagleI clean count: {len(df_eaglei_subset)}")

NOAA clean count: 394824
EagleI clean count: 190289642


In [7]:
import pandas as pd 
df_noaa = pd.read_pickle("noaa_fips_cleaned_sorted.pkl")
df_eaglei_subset = pd.read_pickle("eaglei_subset_clean_sorted.pkl")

In [8]:
# Check that the key columns contain identical values
print("\nKey column verification:")
print(f"NOAA FIPS unique count: {df_noaa['full_fips_code'].nunique()}")
print(f"EagleI FIPS unique count: {df_eaglei_subset['fips_code'].nunique()}")

# Check for values in one but not the other
noaa_fips = set(df_noaa['full_fips_code'].unique())
eaglei_fips = set(df_eaglei_subset['fips_code'].unique())
print(f"FIPS in NOAA but not EagleI: {len(noaa_fips - eaglei_fips)}")
print(f"FIPS in EagleI but not NOAA: {len(eaglei_fips - noaa_fips)}")

# Check dtypes
print("\nDtypes:")
print(f"NOAA full_fips_code: {df_noaa['full_fips_code'].dtype}")
print(f"EagleI fips_code: {df_eaglei_subset['fips_code'].dtype}")


Key column verification:
NOAA FIPS unique count: 3163
EagleI FIPS unique count: 3086
FIPS in NOAA but not EagleI: 87
FIPS in EagleI but not NOAA: 10

Dtypes:
NOAA full_fips_code: object
EagleI fips_code: object


In [9]:
print("\n--- Verifying df_noaa is properly sorted for merge_asof ---")

# Check if full_fips_code is monotonically increasing
fips_sorted = df_noaa['full_fips_code'].is_monotonic_increasing
print(f"1. full_fips_code is monotonically increasing: {fips_sorted}")

if not fips_sorted:
    # Find where the sorting breaks
    diff = df_noaa['full_fips_code'].iloc[1:] < df_noaa['full_fips_code'].iloc[:-1].values
    if diff.any():
        first_violation = diff.idxmax()
        print(f"  Sorting violation found at index {first_violation}:")
        print(f"    {first_violation-1}: {df_noaa['full_fips_code'].iloc[first_violation-1]}")
        print(f"    {first_violation}: {df_noaa['full_fips_code'].iloc[first_violation]}")

# Check if END_DT_UTC is sorted within each FIPS group
group_checks = df_noaa.groupby('full_fips_code')['END_DT_UTC'].apply(lambda x: x.is_monotonic_increasing)
all_groups_sorted = group_checks.all()
print(f"2. END_DT_UTC is sorted within each full_fips_code group: {all_groups_sorted}")

if not all_groups_sorted:
    bad_groups = group_checks[~group_checks].index.tolist()
    print(f"  Found {len(bad_groups)} groups with sorting issues:")
    for i, fips in enumerate(bad_groups[:3]):  # Show first 3 problematic groups
        group = df_noaa[df_noaa['full_fips_code'] == fips]
        print(f"  - FIPS {fips} (size {len(group)}):")
        # Find where the sorting breaks within this group
        diff = group['END_DT_UTC'].iloc[1:] < group['END_DT_UTC'].iloc[:-1].values
        if diff.any():
            first_violation = diff.idxmax()
            print(f"    Out-of-order at index {first_violation}:")
            print(f"      {first_violation-1}: {group.loc[first_violation-1, 'END_DT_UTC']}")
            print(f"      {first_violation}: {group.loc[first_violation, 'END_DT_UTC']}")

# Only proceed with merge if both checks pass
if fips_sorted and all_groups_sorted:
    print("✓ df_noaa is properly sorted for merge_asof")
else:
    print("✗ df_noaa is NOT properly sorted for merge_asof - must fix before merging")
    # Consider adding: exit() or df_noaa = df_noaa.sort_values(...) to auto-fix


--- Verifying df_noaa is properly sorted for merge_asof ---
1. full_fips_code is monotonically increasing: True
2. END_DT_UTC is sorted within each full_fips_code group: True
✓ df_noaa is properly sorted for merge_asof


In [13]:
# 1. Check for null values in key columns
print("\nNull values check:")
print(f"full_fips_code nulls: {df_noaa['full_fips_code'].isnull().sum()}")
print(f"END_DT_UTC nulls: {df_noaa['END_DT_UTC'].isnull().sum()}")

# 2. Verify consistent data types
print("\nData type consistency:")
print("FIPS code sample:", df_noaa['full_fips_code'].head(3).tolist())
print("Timestamp sample:", df_noaa['END_DT_UTC'].head(3).tolist())

# 3. Check string representations of FIPS codes
print("\nFIPS code formatting:")
print("Lengths:", df_noaa['full_fips_code'].str.len().value_counts())
print("Leading zeros sample:", df_noaa['full_fips_code'].str.startswith('0').value_counts())

# 4. Timezone check
print("\nTimezone info:")
print(df_noaa['END_DT_UTC'].dt.tz)


Null values check:
full_fips_code nulls: 0
END_DT_UTC nulls: 0

Data type consistency:
FIPS code sample: ['01001', '01001', '01001']
Timestamp sample: [Timestamp('2014-01-11 12:15:00+0000', tz='UTC'), Timestamp('2014-01-11 12:20:00+0000', tz='UTC'), Timestamp('2014-04-07 18:35:00+0000', tz='UTC')]

FIPS code formatting:
Lengths: full_fips_code
5    394824
Name: count, dtype: int64
Leading zeros sample: full_fips_code
False    353274
True      41550
Name: count, dtype: int64

Timezone info:
UTC


In [22]:
df_noaa['END_DT_UTC'] = pd.to_datetime(df_noaa['END_DT_UTC']).dt.tz_localize(None)
print(df_noaa['END_DT_UTC'].dtype)


datetime64[ns]


In [30]:
df_noaa.sort_values(by=['full_fips_code', 'END_DT_UTC'], inplace=True)

df_noaa['END_DT_UTC'].is_monotonic_increasing


False

In [ ]:
df_eaglei_subset.head(1)

## Successful merge code below
The merge logic using the numpy array for the dataframes of merge_asof worked over pandas df.

I suspect the reason to be that the keys of merging(esp the timezone aware date-time column) contains nano second values which are not sorted by pandas(up to that s.f.) which was causing the sort error in merge. 


In [ ]:
import numpy as np

def prepare_for_merge_asof(df, by_col, on_col):
    """Ensure perfect sorting that merge_asof will accept"""
    # Convert to numpy arrays for precise control
    groups = df[by_col].values
    times = df[on_col].values.astype('datetime64[ns]')
    
    # Create index array
    index = np.arange(len(df))
    
    # Sort first by group, then by time using mergesort
    # This creates a lexsort with group as primary key, time as secondary
    sorted_indices = np.lexsort((times, groups))
    
    # Apply the sorting
    sorted_df = df.iloc[sorted_indices].reset_index(drop=True)
    
    # Final verification
    verify_sorting(sorted_df, by_col, on_col)
    
    return sorted_df

def verify_sorting(df, by_col, on_col):
    """Rigorous sorting verification"""
    current_group = None
    last_time = None
    
    for i in range(len(df)):
        if df[by_col].iloc[i] != current_group:
            current_group = df[by_col].iloc[i]
            last_time = df[on_col].iloc[i]
        else:
            if df[on_col].iloc[i] < last_time:
                raise ValueError(f"Sort violation at index {i}")
            last_time = df[on_col].iloc[i]
    print("✓ Perfect sorting verified")

# Prepare both DataFrames
df_noaa_prepared = prepare_for_merge_asof(df_noaa, 'full_fips_code', 'END_DT_UTC')
df_eaglei_prepared = prepare_for_merge_asof(df_eaglei_renamed, 'full_fips_code', 'END_DT_UTC')

# Perform the merge
try:
    merged_df = pd.merge_asof(
        df_noaa_prepared,
        df_eaglei_prepared,
        on='END_DT_UTC',
        by='full_fips_code',
        direction='forward',
        tolerance=pd.Timedelta('2h')
    )
    print("✅ Merge succeeded after rigorous sorting")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    print("\n⚠️ Implementing manual merge_asof logic as fallback")
    
    # Manual implementation of merge_asof
    merged_records = []
    for fips in df_noaa_prepared['full_fips_code'].unique():
        noaa_group = df_noaa_prepared[df_noaa_prepared['full_fips_code'] == fips]
        eaglei_group = df_eaglei_prepared[df_eaglei_prepared['full_fips_code'] == fips]
        
        for _, noaa_row in noaa_group.iterrows():
            # Find closest matching row in eaglei data
            matches = eaglei_group[
                (eaglei_group['END_DT_UTC'] >= noaa_row['END_DT_UTC']) &
                (eaglei_group['END_DT_UTC'] <= noaa_row['END_DT_UTC'] + pd.Timedelta('2h'))
            ]
            
            if not matches.empty:
                best_match = matches.iloc[0]  # First match due to sorting
                merged_row = {**noaa_row.to_dict(), **best_match.to_dict()}
                merged_records.append(merged_row)
            else:
                merged_records.append(noaa_row.to_dict())
    
    merged_df = pd.DataFrame(merged_records)

In [ ]:
def check_fips_formatting(df, col_name):
    """Verify all FIPS codes are 5-digit strings"""
    print(f"\nChecking FIPS formatting in column '{col_name}':")
    
    # Check length
    length_check = df[col_name].str.len() != 5
    bad_length = df[length_check]
    
    # Check digit-only
    digit_check = ~df[col_name].str.match(r'^\d{5}$', na=False)
    bad_digits = df[digit_check]
    
    # Check leading zeros (first 2 digits should be state code 01-56)
    state_code_check = ~df[col_name].str[:2].isin([f"{i:02d}" for i in range(1,57)])
    bad_state_codes = df[state_code_check]
    
    # Report findings
    if len(bad_length) > 0:
        print(f"  Found {len(bad_length)} entries with wrong length:")
        print(bad_length[col_name].value_counts().head())
    
    if len(bad_digits) > 0:
        print(f"  Found {len(bad_digits)} non-numeric entries:")
        print(bad_digits[col_name].unique())
    
    if len(bad_state_codes) > 0:
        print(f"  Found {len(bad_state_codes)} questionable state codes:")
        print(bad_state_codes[col_name].str[:2].value_counts())
    
    # Return clean mask
    return ~(length_check | digit_check | state_code_check)

# Run checks on both datasets
noaa_clean_mask = check_fips_formatting(df_noaa, 'full_fips_code')
eaglei_clean_mask = check_fips_formatting(df_eaglei_subset, 'fips_code')

# Show clean/dirty counts
print(f"\nNOAA clean/dirty counts: {noaa_clean_mask.sum()} clean, {len(df_noaa)-noaa_clean_mask.sum()} dirty")
print(f"EagleI clean/dirty counts: {eaglei_clean_mask.sum()} clean, {len(df_eaglei_subset)-eaglei_clean_mask.sum()} dirty")

# Optional: Filter to only clean FIPS codes
df_noaa_clean = df_noaa[noaa_clean_mask].copy()
df_eaglei_clean = df_eaglei_subset[eaglei_clean_mask].copy()

In [8]:
# Check actual types of values (not just dtypes)
def check_value_types(series):
    return series.apply(lambda x: type(x)).value_counts()

print("NOAA value types:")
print(check_value_types(df_noaa['full_fips_code']))

print("EagleI value types:")
print(check_value_types(df_eaglei_subset['fips_code']))

NOAA value types:
full_fips_code
<class 'str'>    394824
Name: count, dtype: int64
EagleI value types:
fips_code
<class 'str'>    190289642
Name: count, dtype: int64


In [9]:
# 1. Verify time column formats
print("NOAA END_DT_UTC sample:", df_noaa['END_DT_UTC'].head(1).values)
print("EagleI EAGLEI_DT_UTC sample:", df_eaglei_subset['EAGLEI_DT_UTC'].head(1).values)

# 2. Confirm no missing keys
print("NOAA missing FIPS:", df_noaa['full_fips_code'].isna().sum())
print("EagleI missing FIPS:", df_eaglei_subset['fips_code'].isna().sum())

# 3. Check memory usage (for large merges)
print("\nMemory usage:")
print("NOAA:", df_noaa.memory_usage(deep=True).sum()/1e6, "MB")
print("EagleI:", df_eaglei_subset.memory_usage(deep=True).sum()/1e6, "MB")

NOAA END_DT_UTC sample: ['2014-01-11T12:15:00.000000000']
EagleI EAGLEI_DT_UTC sample: ['2014-11-01T16:15:00.000000000']
NOAA missing FIPS: 0
EagleI missing FIPS: 0

Memory usage:
NOAA: 1011.52597 MB
EagleI: 14842.592076 MB


In [12]:
df_noaa.to_pickle("noaa_fips_cleaned_sorted.pkl")
df_eaglei_subset.to_pickle("eaglei_subset_clean_sorted.pkl")

In [14]:
def verify_merge_asof_sort(df, by_col, on_col):
    """Verify DataFrame is properly sorted for merge_asof"""
    # 1. Check primary key is monotonic
    if not df[by_col].is_monotonic_increasing:
        print(f"Primary sort key '{by_col}' is NOT monotonic!")
        return False
    
    # 2. Check secondary key is sorted within primary key groups
    group_status = df.groupby(by_col)[on_col].agg(lambda x: x.is_monotonic_increasing)
    if not group_status.all():
        bad_groups = group_status[~group_status].index.tolist()
        print(f"Found {len(bad_groups)} groups where '{on_col}' isn't sorted")
        print("First 3 bad groups:", bad_groups[:3])
        return False
    
    # 3. Check no NaN values in sort keys
    if df[[by_col, on_col]].isna().any().any():
        print("NaN values found in sort keys!")
        return False
    
    return True

# Verify both DataFrames
print("NOAA sort valid:", verify_merge_asof_sort(df_noaa, 'full_fips_code', 'END_DT_UTC'))
print("EagleI sort valid:", verify_merge_asof_sort(df_eaglei_subset, 'fips_code', 'EAGLEI_DT_UTC'))

NOAA sort valid: True
EagleI sort valid: True


In [15]:
def is_actually_sorted(df, keys):
    """Absolute verification by re-sorting and comparing"""
    sorted_copy = df.sort_values(by=keys, kind='mergesort').reset_index(drop=True)
    return df.reset_index(drop=True).equals(sorted_copy)

print("NOAA actually sorted:", 
      is_actually_sorted(df_noaa, ['full_fips_code', 'END_DT_UTC']))
print("EagleI actually sorted:", 
      is_actually_sorted(df_eaglei_subset, ['fips_code', 'EAGLEI_DT_UTC']))

NOAA actually sorted: True
EagleI actually sorted: True


In [16]:
print("NOAA timezone:", df_noaa['END_DT_UTC'].dt.tz)
print("EagleI timezone:", df_eaglei_subset['EAGLEI_DT_UTC'].dt.tz)

NOAA timezone: UTC
EagleI timezone: UTC


In [19]:
import psutil

logical_cores = psutil.cpu_count(logical=True)     # Threads (logical CPUs)
physical_cores = psutil.cpu_count(logical=False)   # Physical cores

print(f"Logical cores (threads): {logical_cores}")
print(f"Physical cores: {physical_cores}")


Logical cores (threads): 20
Physical cores: 20


In [1]:
conda list | grep dask

dask                      2023.8.1                 pypi_0    pypi
dask-core                 2025.2.0        py312h06a4308_0  
dask-expr                 2.0.0           py312h06a4308_0  

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip list | grep pandas

pandas                    2.2.1
Note: you may need to restart the kernel to use updated packages.
